In [1]:
#import libraries
import cv2
import numpy as np
import pickle as pkl
import tensorflow as tf
import tensorflow_hub as hub

from matplotlib import pyplot as plt
from pathlib import Path

In [2]:
# Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
#Load Model
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

In [4]:
# Draw edges
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [5]:
# Draw the keypoints on the human subject
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

In [6]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

In [7]:
# Draw connectors(lines from joints)
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

In [8]:
#Run this just to view the video with the keypoint and edges
cap = cv2.VideoCapture(r"C:\Users\ayesh\Downloads\Yaish\Fullfoul\selfshot_video_1.mp4")
while cap.isOpened():
    ret, frame = cap.read()
        
    if not ret:
        break
        
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    # Render keypoints 
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)
    
    cv2.imshow('Movenet Multipose', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [9]:
#visualization
keypoints_with_scores

array([[[0.4466518 , 0.8612912 , 0.32815337],
        [0.43496653, 0.8580613 , 0.36712667],
        [0.43250212, 0.8611075 , 0.40836304],
        [0.41552076, 0.86621284, 0.46887434],
        [0.40377066, 0.8816418 , 0.4177208 ],
        [0.4478281 , 0.88256806, 0.6443066 ],
        [0.42883858, 0.95933545, 0.44161174],
        [0.5461881 , 0.8564455 , 0.5259613 ],
        [0.55058146, 0.9550271 , 0.15743777],
        [0.5798087 , 0.8034275 , 0.32677203],
        [0.63623405, 0.9242259 , 0.09693617],
        [0.6640478 , 0.9469134 , 0.35237807],
        [0.6678417 , 0.9929147 , 0.6908403 ],
        [0.8372962 , 0.8682291 , 0.1604816 ],
        [0.83449996, 0.98304504, 0.0901176 ],
        [0.9654872 , 0.86400414, 0.04105417],
        [0.972891  , 0.9850193 , 0.04833445]],

       [[0.45305008, 0.7461603 , 0.21939935],
        [0.4370154 , 0.7451717 , 0.23444988],
        [0.43648058, 0.74688786, 0.2825412 ],
        [0.41049808, 0.7275561 , 0.39465514],
        [0.41217992, 0.72689563,

In [11]:
#visualization
keypoints_with_scores.shape

(6, 17, 3)

In [13]:
# Store the extracted foul/nonfoul keypoints in relevant path
data = []

# for idx, file in enumerate(Path(r"C:\Users\ayesh\Downloads\Yaish\Foul").rglob("*.mp4"), start = 1):
for idx, file in enumerate(Path(r"C:\Users\ayesh\Downloads\Yaish\No Foul").rglob("*.mp4"), start = 1):
#     if idx <= 150:
#         continue
    
    video = []
    cap = cv2.VideoCapture(str(file))

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Resize image
        img = frame.copy()
        img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
        input_img = tf.cast(img, dtype=tf.int32)

        # Detection section
        results = movenet(input_img)
        keypoints_with_scores = results['output_0'].numpy()[:,:2,:51].reshape((2,17,3))
        video.append(keypoints_with_scores)
        
    cap.release()
    data.append(np.array(video))
    print(idx)
    
    if idx == 150:
        break

with open(r"C:\Users\ayesh\Downloads\Yaish\No_Foul.pkl", "wb") as file:
    pkl.dump(data,file)

1
2
3
4
5
6
7
8
9
10
11
12
13


In [14]:
# visualization
print(f"{len(data)}\n\t=>{data[0].shape}")

13
	=>(45, 2, 17, 3)


In [15]:
# visualization
keypoints_with_scores.shape

(2, 17, 3)